Modules

In [11]:
import json
import os
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup as bs

Constantes

In [12]:
PATH_DATA = r"C:\Users\ahoun\Documents\Personnel\Nuku Data\Real-estate-prices-etl\Data"

Application

In [13]:
url = "https://www.immoscout24.ch/en/house/buy/city-bern?pn=1&r=40"
response = requests.get(url)

In [14]:
html = response.content

In [15]:
soup = bs(html, "html")

In [17]:
descs = soup.find_all('div', {'class': "Box-cYFBPY Flex-feqWzG MetaBody-iCkzuU JVKAR dCDRxm dUKbXG"})

data = {'data': [
                ]
        }

for i,des in enumerate(descs):
    content = {'id': i,
                'description': None,
                'room': None,
                'surface': None,
                'price': None
                }
    content['description'] = des.find('p').get_text()


    list_reste = des.find('h3').get_text(strip=True).split('CHF')
    if len(list_reste) == 2:
        price = list_reste[1]
        price = price.replace('CHF ', '')
        price = price.replace('.—', '')
        price = price.replace(',', '').strip()
    else:
        price = None
        

    number_room = list_reste[0].split(',')[0]
    number_room = number_room.replace('rooms', '')
    
    surface = list_reste[0].split(',')[1]
    surface = surface.replace('m²', '')
    
    
    content['room'] = number_room.strip()
    content['surface'] = surface.strip()
    content['price'] = price
    data['data'].append(content)


data

{'data': [{'id': 0,
   'description': "Située à quelques minutes du centre-ville de Neuchâtel, Corcelles est une charmante commune parfaitement implantée entre vignes et forêts. Disposant déjà d'un grand nombre de services, sa proximité avec Neuchâtel en fait un village prisé par des habitants désireux de maintenir une excellente qualité de vie loin des nuisances citadines.C'est au cœur de cette commune, sur un terrain légèrement en pente, que seront prochainement érigées les 4 villas proposées à la vente. En retrait de la Rue des Préels, l'une des principales du village, votre futu",
   'room': '4.5',
   'surface': '150',
   'price': '1520760'},
  {'id': 1,
   'description': "Située à quelques minutes du centre-ville de Neuchâtel, Corcelles est une charmante commune parfaitement implantée entre vignes et forêts. Disposant déjà d'un grand nombre de services, sa proximité avec Neuchâtel en fait un village prisé par des habitants désireux de maintenir une excellente qualité de vie loin d

In [18]:
if os.path.exists(PATH_DATA):
    print('Le dossier Data existe.')
else: 
    os.mkdir(PATH_DATA)
    print('Le dossier a été créé avec succès.')

Le dossier Data existe.


In [19]:
# Nom du fichier de sortie
page = 1
nom_fichier = rf"{PATH_DATA}/data_page{page}.json"

# Sauvegarder le dictionnaire en JSON
with open(nom_fichier, "w", encoding="utf-8") as fichier:
    json.dump(data, fichier, ensure_ascii=False)

In [28]:
df = pd.read_json(nom_fichier, orient='split')
df

,id,description,room,surface,price
0,0,Située à quelques minutes du centre-ville de N...,4.5,150,1520760.0
1,1,Située à quelques minutes du centre-ville de N...,5.5,180,1822290.0
2,2,Modernes Doppeleinfamilienhaus mit Studio in S...,5.5,360,1730520.0
3,3,Nouvelle promotion de villas dans un magnifiqu...,6.5,210,1520760.0
4,4,Nouvelle promotion de villas dans un magnifiqu...,6.5,240,1520760.0
5,5,Nouvelle promotion de villas dans un magnifiqu...,6.5,240,1560090.0
6,6,Nouvelle promotion de villas dans un magnifiqu...,6.5,220,1520760.0
7,7,Nouvelle promotion de villas dans un magnifiqu...,6.5,240,1520760.0
8,8,Nouvelle promotion de villas dans un magnifiqu...,6.5,240,1560090.0
9,9,"In Hofstetten SO, an herrlicher Aussichtslage ...",5.5,310,2431900.0


In [ ]:
i = 1
url = f"https://www.immoscout24.ch/en/house/buy/city-bern?r=40"
response = requests.get(url)
list_des = []
list_prices = []

while response.status_code == 200 :
# while i < 20:
    url = f"https://www.immoscout24.ch/en/house/buy/city-bern?pn={i}&r=40"
    response = requests.get(url)
    # print(url)
    # print(response.status_code)
    html = response.content
    soup = bs(html, "html")
    
    description = soup.find_all('div', {'class': "Text__TextStyled-fiIwWW Hxaps"})
    for des in description:
        # print(des.get_text())
        list_des.append(des.get_text())
        
    pattern = re.compile(r"CHF [\d,]+", re.IGNORECASE)
    prices = soup.find_all('span', text=pattern)
    for price in prices:
        # print(price.get_text())
        text = price.get_text()
        print(text)
        if text == '':
            list_prices.append('')
        else:
            text = text.replace('CHF', '')
            text = text.replace('.—', '')
            list_prices.append(text)
    
    i += 1



    

df = pd.DataFrame(
                  {'Description' : list_des,
                   'Prix': list_prices}  
)
df.head()

In [ ]:
len(list_des), len(list_prices)

In [ ]:
list_des

In [ ]:
list_prices

In [ ]:
df.info()